In [1]:
import numpy as np
import pickle
import gdown
import os
import torch
from sklearn.model_selection import train_test_split
from ipywidgets import IntProgress
from IPython.display import display
import torch.nn as nn
from torch.utils.data import TensorDataset

In [5]:
with open('bases.npy', 'rb') as f:
    data_val = np.load(f)
    val_label = np.load(f)
    data_train = np.load(f)
    train_label = np.load(f)

### Tensor dataset

In [6]:

my_datatrain = TensorDataset(torch.Tensor(data_train),torch.Tensor(train_label)) 
my_dataval = TensorDataset(torch.Tensor(data_val),torch.Tensor(val_label))

## Arquitectura red recurrente

In [7]:
class RNN(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers,num_classes):
        super(RNN, self).__init__()
        self.num_layers=num_layers
        self.hidden_size=hidden_size

        self.rnn=nn.GRU(input_size,hidden_size,num_layers,batch_first=True)
        self.fc=nn.Sequential(nn.Linear(hidden_size,64),
                                nn.ReLU(),
                                nn.Linear(64,32),
                                nn.ReLU(),
                                nn.Linear(32,16),
                                nn.ReLU(),
                                nn.Linear(16,num_classes)
                            )

    
    def forward(self, x,gpu):
        if gpu== True:

            h0=torch.zeros(self.num_layers,x.size(0),self.hidden_size).cuda()
        else:
            h0=torch.zeros(self.num_layers,x.size(0),self.hidden_size)

        out,_=self.rnn(x,h0)
        out=out[:,-1,:]
        out=self.fc(out)
        return out
 


In [9]:
input_size=128
hidden_size=200
num_classes=3
num_epoch=100
batch_size=32
lr=1e-4
num_layers=4
    

model=RNN(input_size,hidden_size,num_layers,num_classes)  
criterion= nn.CrossEntropyLoss() # Probar con Cross entropy
lr=1e-3
optimizer=torch.optim.Adam(model.parameters(),lr=lr) # Probar con Sgd



current_loss=0
all_losses=[]
plot_step, print_step=1000,5000
num_epoch=100
batch_size=32
use_gpu=False

train_loader = torch.utils.data.DataLoader(my_datatrain, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=use_gpu)
val_loader = torch.utils.data.DataLoader(my_dataval, batch_size=len(my_dataval), shuffle=False, pin_memory=use_gpu)
num_total_steps=len(train_loader)

if use_gpu:
    model.cuda()

for epoch in range(num_epoch):
    acumulado=0
    train_acc_count=0
    val_acc_count=0

    cumulative_train_corrects=0

    model.train()   
    for i, (x_batch, y_batch) in enumerate(train_loader):
        largo=len(x_batch)
        # print(np.shape(x_batch))
        if use_gpu:
            x_batch = x_batch.cuda()
            y_batch = y_batch.cuda()

        outputs=model(x_batch,use_gpu)

        if use_gpu:
            y_batch = y_batch.type(torch.LongTensor).cuda()
        else:
            y_batch = y_batch.type(torch.LongTensor)

        class_prediction = torch.argmax(outputs, axis=1).long()
        train_acc_count += y_batch.shape[0]
        cumulative_train_corrects += (y_batch == class_prediction).sum().item()

        loss=criterion(outputs, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acumulado+=loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epoch}], Loss:{acumulado/num_total_steps:.4f}, acc train: {cumulative_train_corrects / train_acc_count:.4f}")

    model.eval()
    with torch.no_grad():
        val_acc_count=0
        cumulative_val_corrects=0
        for x_val, y_val in val_loader:
            if use_gpu:
                x_val = x_val.cuda()
                y_val = y_val.cuda()

            y_predicted = model(x_val,use_gpu)
            class_prediction = torch.argmax(y_predicted, axis=1).long()
            val_acc_count += y_val.shape[0]
            cumulative_val_corrects += (y_val == class_prediction).sum().item()

    print(f"Epoch [{epoch+1}/{num_epoch}], acc val: {cumulative_val_corrects / val_acc_count:.4f}")

KeyboardInterrupt: 